## Plain vs finegrained

¿Cuánta ganancia tenemos en la detección?



In [1]:
%load_ext autoreload
%autoreload 2

from hatedetection import load_datasets
import glob
import json

train_dataset, dev_dataset, test_dataset = load_datasets(add_body=True)

INFO: Pandarallel will run on 24 workers.
INFO: Pandarallel will use Memory file system to transfer data between the main process and workers.


Obs

`text` hace referencia al contexto del tweet del diario
`title` al título de la noticia (extraído del artículo)

In [2]:
!ls ../evaluations/

beto_fine_body.json		  betonews_plain_none.json
beto_fine_none_weighted.json	  betonews_plain_text.json
beto_fine_title.json		  beto_plain_none.json
beto_fine_title_nonweighted.json  beto_plain_text+body.json
betonews_fine_body.json		  beto_plain_text.json
betonews_fine_none.json		  dev
betonews_fine_text.json		  old
betonews_fine_title.json


In [3]:
evaluations = {
}
for model_name, path in [
    ('plain', "../evaluations/betonews_plain_text.json"),   
    ('fine', "../evaluations/betonews_fine_text.json"),   
    ]:

    with open(path) as f:
        evaluations[model_name] = json.load(f)

for key, evals in evaluations.items():
    print(f"We have {len(evals['metrics'])} {key} evaluations")


We have 10 plain evaluations
We have 10 fine evaluations


In [4]:
import pandas as pd

df_plain = pd.DataFrame(evaluations["plain"]["metrics"])

df_plain.mean()

test_loss                    0.615216
test_accuracy                0.910138
test_f1                      0.697287
test_macro_f1                0.822260
test_precision               0.748445
test_recall                  0.653255
test_runtime                33.804930
test_samples_per_second    335.544700
test_steps_per_second       20.973500
dtype: float64

In [5]:
df_fine = pd.DataFrame(evaluations["fine"]["metrics"])

df_fine.mean()

test_loss                    0.068422
test_calls_f1                0.684652
test_women_f1                0.420590
test_lgbti_f1                0.482327
test_racism_f1               0.720456
test_class_f1                0.511453
test_politics_f1             0.624763
test_disabled_f1             0.608863
test_appearance_f1           0.766254
test_criminal_f1             0.699382
test_mean_f1                 0.613193
test_mean_precision          0.702100
test_mean_recall             0.550936
test_hate_precision          0.759889
test_hate_recall             0.665943
test_hate_f1                 0.709637
test_runtime                34.542440
test_samples_per_second    328.509200
test_steps_per_second       10.281300
dtype: float64

¿Es más grande en promedio? Hagamos un Mann-Whitney U

In [6]:
import scipy.stats

plain_f1 = df_plain["test_f1"]
fine_f1 = df_fine["test_hate_f1"]

scipy.stats.mannwhitneyu(plain_f1, fine_f1)


MannwhitneyuResult(statistic=0.0, pvalue=9.133589555477501e-05)